In [51]:
%load_ext lab_black
%load_ext autoreload
%autoreload 2

The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
from pathlib import Path
from puzzle import Puzzle

# from skimage.graph import route_through_array

In [47]:
puz = Puzzle("tests.txt")
puz

Puzzle(fname='tests.txt')

In [50]:
p = puz.expand_grid()


big_grid

array([[1, 1, 6, ..., 2, 8, 6],
       [1, 3, 8, ..., 1, 2, 6],
       [2, 1, 3, ..., 7, 6, 3],
       ...,
       [7, 5, 6, ..., 5, 2, 8],
       [5, 6, 4, ..., 4, 1, 9],
       [6, 7, 5, ..., 4, 7, 9]])

In [41]:
puz.part_1()

40

In [44]:
grid = puz.maze
grid

np.where(grid == 9, 1, grid + 1)

array([[2, 2, 7, 4, 8, 6, 2, 8, 5, 3],
       [2, 4, 9, 2, 4, 8, 4, 7, 8, 3],
       [3, 2, 4, 7, 6, 2, 2, 4, 3, 9],
       [4, 7, 1, 5, 1, 4, 2, 6, 7, 1],
       [8, 5, 7, 4, 5, 2, 8, 2, 2, 2],
       [2, 4, 2, 1, 2, 3, 9, 2, 4, 8],
       [2, 4, 6, 1, 1, 2, 3, 5, 3, 2],
       [4, 2, 3, 6, 5, 3, 2, 7, 4, 1],
       [2, 3, 1, 4, 2, 4, 9, 6, 3, 2],
       [3, 4, 2, 2, 1, 5, 5, 6, 9, 2]])

In [39]:
route[0, 1:]

array([1, 2, 2, 2, 2, 2, 2, 2, 3, 3, 4, 5, 5, 6, 7, 8, 8, 9])

In [34]:
puz.maze

array([[1, 1, 6, 3, 7, 5, 1, 7, 4, 2],
       [1, 3, 8, 1, 3, 7, 3, 6, 7, 2],
       [2, 1, 3, 6, 5, 1, 1, 3, 2, 8],
       [3, 6, 9, 4, 9, 3, 1, 5, 6, 9],
       [7, 4, 6, 3, 4, 1, 7, 1, 1, 1],
       [1, 3, 1, 9, 1, 2, 8, 1, 3, 7],
       [1, 3, 5, 9, 9, 1, 2, 4, 2, 1],
       [3, 1, 2, 5, 4, 2, 1, 6, 3, 9],
       [1, 2, 9, 3, 1, 3, 8, 5, 2, 1],
       [2, 3, 1, 1, 9, 4, 4, 5, 8, 1]])

In [ ]:
puz.part_2()

## Black format my final answer

In [52]:
from pathlib import Path
from dataclasses import dataclass, field
import numpy as np
from skimage.graph import route_through_array


T1_ANS = 40
T2_ANS = 315


@dataclass
class Puzzle:
    fname: str

    def __post_init__(self):
        raw = Path(self.fname).open().readlines()  # or read() for a big block of text
        maze = np.array([list(row.strip()) for row in raw], dtype=int)
        self.maze = maze
        return

    def expand_grid(self):
        """original grid is copied into a 5x5 mega grid.
        Each time copy horizontally or vertically adds one, but 9s wrap back to 1s

        These are the amounts added BEFORE wrapping:

        0 1 2 3 4
        1 2 3 4 5
        2 3 4 5 6
        3 4 5 6 7
        4 5 6 7 8
        """

        def grow(grid):
            return np.where(grid == 9, 1, grid + 1)

        pos_grids = {0: self.maze}

        for n in range(1, 9):
            pos_grids[n] = grow(pos_grids[n - 1])

        big_grid = []
        # Build horizontal rows of 5, then vstack them at the end
        for start in range(5):
            big_grid.append(np.hstack([pos_grids[i] for i in range(start, start + 5)]))

        return np.vstack(big_grid)

    def get_lowest_risk_route(self, maze=None):
        if maze is None:
            maze = self.maze
        end = [dim - 1 for dim in maze.shape]
        route, score = route_through_array(maze, (0, 0), end, fully_connected=False)
        route = np.stack(route, axis=-1)
        return maze[route[0, 1:], route[1, 1:]].sum()

    def part_1(self):
        return self.get_lowest_risk_route()

    def part_2(self):
        big_maze = self.expand_grid()
        return self.get_lowest_risk_route(big_maze)


def run_tests(p1_ans=T1_ANS, p2_ans=T2_ANS, fname="tests.txt"):
    puz = Puzzle(fname)
    t1 = puz.part_1()
    assert t1 == p1_ans, f"Test 1 failed. Got {t1} instead of {p1_ans}"

    if p2_ans is not None:
        t2 = puz.part_2()
        assert t2 == p2_ans, f"Test 2 failed. Got {t2} instead of {p2_ans}"

    print("All tests passed.")
    return


if __name__ == "__main__":
    run_tests()

    puz = Puzzle("inputs.txt")

    p1 = puz.part_1()
    print("Part 1:", p1)

    if T2_ANS is not None:
        p2 = puz.part_2()
        print("Part 2:", p2)

All tests passed.
Part 1: 696
Part 2: 2952
